In [1]:
import pandas as pd
import os
import geopandas as gpd
from pymongo import MongoClient
import requests
import json 
from dotenv import load_dotenv
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
client = MongoClient("mongodb://localhost/ironhack")
db = client.get_database()